In [19]:
import pandas as pd
import numpy as np
from flask import Flask, jsonify, request
import pickle
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report




In [20]:

ruta_archivo = r"C:\\Users\\orlan\\Documents\\Proyecto Machine Learning\\datos_demograficos.csv"


data = pd.read_csv(ruta_archivo, delimiter=';')


data['Total'] = data['Total'].str.replace(',', '.').str.replace('..', '.')
data['Total'] = pd.to_numeric(data['Total'], errors='coerce')
data = data.dropna(subset=['Total'])

data['ID_Cliente'] = np.arange(1, len(data) + 1)
data['Ingresos'] = np.random.normal(30000, 10000, len(data)).clip(min=15000)
data['Compro_Seguro'] = np.random.choice([0, 1], len(data), p=[0.7, 0.3])
data['Edad'] = np.random.choice(['18-25', '26-35', '36-45', '46-55', '56-65', '65+'], len(data))
data['Estado civil'] = np.random.choice(['Soltero', 'Casado', 'Divorciado', 'Viudo'], len(data))
data['Sexo'] = np.random.choice(['M', 'F'], len(data))
data['Tamaño_Hogar'] = np.random.choice([1, 2, 3, 4, 5], len(data))
data['Nivel_Educativo'] = np.random.choice(['Primaria', 'Secundaria', 'Universitario', 'Postgrado'], len(data))


data = pd.get_dummies(data, columns=['Edad', 'Sexo', 'Estado civil', 'Nivel_Educativo'], drop_first=True)


test_data = data.sample(frac=0.2, random_state=42)


print("Columnas en el DataFrame:", data.columns)
print("Número de filas en test_data:", len(test_data))



Columnas en el DataFrame: Index(['Periodo', 'Total', 'ID_Cliente', 'Ingresos', 'Compro_Seguro',
       'Tamaño_Hogar', 'Edad_26-35', 'Edad_36-45', 'Edad_46-55', 'Edad_56-65',
       'Edad_65+', 'Sexo_M', 'Estado civil_Divorciado', 'Estado civil_Soltero',
       'Estado civil_Viudo', 'Nivel_Educativo_Primaria',
       'Nivel_Educativo_Secundaria', 'Nivel_Educativo_Universitario'],
      dtype='object')
Número de filas en test_data: 263


In [21]:

X = data.drop(['Compro_Seguro', 'Periodo'], axis=1)  
y = data['Compro_Seguro']


print(X.dtypes)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(class_report)


with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)










Total                            float64
ID_Cliente                         int32
Ingresos                         float64
Tamaño_Hogar                       int32
Edad_26-35                          bool
Edad_36-45                          bool
Edad_46-55                          bool
Edad_56-65                          bool
Edad_65+                            bool
Sexo_M                              bool
Estado civil_Divorciado             bool
Estado civil_Soltero                bool
Estado civil_Viudo                  bool
Nivel_Educativo_Primaria            bool
Nivel_Educativo_Secundaria          bool
Nivel_Educativo_Universitario       bool
dtype: object
Accuracy: 0.6136363636363636
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.85      0.74       171
           1       0.39      0.17      0.24        93

    accuracy                           0.61       264
   macro avg       0.52      0.51      0.49       264
weight

In [22]:

app = Flask(__name__)


with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)


print("Número de filas en test_data (inicialización):", len(test_data))






Número de filas en test_data (inicialización): 263


In [23]:
@app.route('/datos', methods=['GET'])
def obtener_todos_los_datos():
    return jsonify(data.to_dict(orient='records'))

@app.route('/datos/train', methods=['GET'])
def obtener_datos_train():
    return jsonify(data.to_dict(orient='records'))

@app.route('/datos/test', methods=['GET'])
def obtener_datos_test():
    try:
        print("Número de filas en test_data (servidor):", len(test_data))
        print(test_data.head())  
        return jsonify(test_data.to_dict(orient='records'))
    except Exception as e:
        print("Error en la ruta /datos/test:", str(e))
        return jsonify({'error': str(e)}), 500

@app.route('/datos/<int:indice>', methods=['GET'])
def obtener_datos_observacion(indice):
    if 0 <= indice < len(data):
        return jsonify(data.iloc[indice].to_dict())
    return jsonify({'error': 'Índice fuera de rango'}), 404

@app.route('/query', methods=['POST'])
def realizar_consulta_query():
    filtro = request.json.get('filtro')
    datos_filtrados = data.query(filtro)
    return jsonify(datos_filtrados.to_dict(orient='records'))

@app.route('/predict', methods=['POST'])
def realizar_prediccion_modelo():
    try:
        observacion = request.json
        df_observacion = pd.DataFrame([observacion])
        
        
        df_observacion = df_observacion.reindex(columns=X_train.columns, fill_value=0)
        
        print("Datos de observación recibidos para predicción:", df_observacion)
        prediccion = model.predict(df_observacion)
        print("Predicción realizada:", prediccion)
        return jsonify({'prediccion': prediccion.tolist()})
    except Exception as e:
        print("Error durante la predicción:", str(e))
        return jsonify({'error': str(e)}), 400

@app.route('/columnas_modelo', methods=['GET'])
def obtener_columnas_modelo():
    return jsonify(list(X_train.columns))






In [24]:
from threading import Thread

def run_flask():
    app.run(debug=True, port=5000, use_reloader=False)

flask_thread = Thread(target=run_flask)
flask_thread.start()


 * Serving Flask app '__main__'


 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
